## Produce daily Origin-Destiniation trip matrix for NYC resident boroughs to NYC metro region municipalities

##### Output is daily csv o-d matrices, O = NYC total or NYC borough and D = NYC metro region municipality (county subdivision - census designated place geography)

In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [2]:
from geo import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [3]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# read nyc origin cbgs
cbg_nyc = pd.read_csv(f'../data/nyc_cbg.csv')
dest_subpl = pd.read_excel(f'../data/31cr_cbg-subpl_xwalk.xlsx')

In [6]:
dest_subpl['dest_cbg'] = dest_subpl['dest_cbg'].astype(str)
dest_subpl['dest_cbg'] = dest_subpl['dest_cbg'].str.zfill(12)
dest_subpl['dest_cbg'] = dest_subpl['dest_cbg'].astype(str)

In [7]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

In [8]:
# set date variables, in this case, a test month for 2 different years
y = "2020"
y0 = "2019"

monthList =["01","02","03","04","05","06","07","08","09","10","11","12"]
dayNumList =[31, 28, 31, 30, 31, 30, 31,31,30,31,30,31] 
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

In [9]:
## Test month - individual days for year-over-year comparison

frames = []
for m in range(0,12):
    for d in range(0,dayNumList[m]):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y0}/{monthList[m]}/{dayList[d]}/{y0}-{monthList[m]}-{dayList[d]}-social-distancing.csv.gz','rb') as f:
            print(f'{y0}-{monthList[m]}-{dayList[d]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json destination cbgs
            df = sgpy.unpack_json_and_merge(df, json_column='destination_cbgs', key_col_name='destination_cbg', value_col_name='dest_cbg_count')
            
            #select destinations in the region only & code in subplace ID
            df = pd.merge(dest_subpl,df,left_on="dest_cbg", right_on="destination_cbg", how="inner")
            
            ##Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str) #clean origin cbg
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            
            #id 31cr counties and all others outside
            df['dest_stco_reg'] = df['destination_cbg'].str[:5]
            df.loc[~df['dest_stco_reg'].isin(stco),'dest_stco_reg'] = 'O31CR' 
            df['dest_sub'] = df['dest_stco_reg'].map(sub).fillna('O31CR')
             
            #Make new table with select columns
            dff = df[['date_y-m-d','orig_stco','dest_sub','dest_subpl','dest_cbg_count']]
            frames.append(dff)
            

2019-09-01
2019-09-02
2019-09-03
2019-09-04
2019-09-05
2019-09-06
2019-09-07
2019-09-08
2019-09-09
2019-09-10
2019-09-11
2019-09-12
2019-09-13
2019-09-14
2019-09-15
2019-09-16
2019-09-17
2019-09-18
2019-09-19
2019-09-20
2019-09-21
2019-09-22
2019-09-23
2019-09-24
2019-09-25
2019-09-26
2019-09-27
2019-09-28
2019-09-29
2019-09-30
2019-10-01
2019-10-02
2019-10-03
2019-10-04
2019-10-05
2019-10-06
2019-10-07
2019-10-08
2019-10-09
2019-10-10
2019-10-11
2019-10-12
2019-10-13
2019-10-14
2019-10-15
2019-10-16
2019-10-17
2019-10-18
2019-10-19
2019-10-20
2019-10-21
2019-10-22
2019-10-23
2019-10-24
2019-10-25
2019-10-26
2019-10-27
2019-10-28
2019-10-29
2019-10-30
2019-10-31
2019-11-01
2019-11-02
2019-11-03
2019-11-04
2019-11-05
2019-11-06
2019-11-07
2019-11-08
2019-11-09
2019-11-10
2019-11-11
2019-11-12
2019-11-13
2019-11-14
2019-11-15
2019-11-16
2019-11-17
2019-11-18
2019-11-19
2019-11-20
2019-11-21
2019-11-22
2019-11-23
2019-11-24
2019-11-25
2019-11-26
2019-11-27
2019-11-28
2019-11-29
2019-11-30

In [10]:
df_dest = pd.concat(frames)

In [11]:
df_dest_bor = pd.pivot_table(df_dest,values=['dest_cbg_count'],index=['date_y-m-d','orig_stco'],columns=['dest_sub','dest_subpl'],aggfunc=np.sum,\
                         fill_value=0,margins=True)

In [12]:
df_dest_nyc = pd.pivot_table(df_dest,values=['dest_cbg_count'],index=['date_y-m-d'],columns=['dest_sub','dest_subpl'],aggfunc=np.sum,\
                         fill_value=0,margins=True)

In [13]:
df_dest_bor.to_csv(f'output/Destinations/dest_coxsubpl_daily_09-12-20{y0}.csv')
df_dest_nyc.to_csv(f'output/Destinations/dest_nycxsubpl_daily_09-12-20{y0}.csv')

### 2020

In [14]:
#adjusted for February leap year
dayNumList =[31, 29, 31, 30, 31, 30, 31,31,30,31,30,31] 

In [15]:
## September only
frames = []
for m in range(0,9):
    for d in range(0,dayNumList[m]):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{monthList[m]}/{dayList[d]}/{y}-{monthList[m]}-{dayList[d]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{monthList[m]}-{dayList[d]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json destination cbgs
            df = sgpy.unpack_json_and_merge(df, json_column='destination_cbgs', key_col_name='destination_cbg', value_col_name='dest_cbg_count')
            
            #select destinations in the region only & code in subplace ID
            df = pd.merge(dest_subpl,df,left_on="dest_cbg", right_on="destination_cbg", how="inner")
            
            ##Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str) #clean origin cbg
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            
            #id 31cr counties and all others outside
            df['dest_stco_reg'] = df['destination_cbg'].str[:5]
            df.loc[~df['dest_stco_reg'].isin(stco),'dest_stco_reg'] = 'O31CR' 
            df['dest_sub'] = df['dest_stco_reg'].map(sub).fillna('O31CR')
             
            #Make new table with select columns
            dff = df[['date_y-m-d','orig_stco','dest_sub','dest_subpl','dest_cbg_count']]
            frames.append(dff)

2020-09-01
2020-09-02
2020-09-03
2020-09-04
2020-09-05
2020-09-06
2020-09-07
2020-09-08
2020-09-09
2020-09-10
2020-09-11
2020-09-12
2020-09-13
2020-09-14
2020-09-15
2020-09-16
2020-09-17
2020-09-18
2020-09-19
2020-09-20
2020-09-21
2020-09-22
2020-09-23
2020-09-24
2020-09-25
2020-09-26
2020-09-27
2020-09-28
2020-09-29
2020-09-30


In [16]:
df_dest = pd.concat(frames)

In [17]:
df_dest_bor = pd.pivot_table(df_dest,values=['dest_cbg_count'],index=['date_y-m-d','orig_stco'],columns=['dest_sub','dest_subpl'],aggfunc=np.sum,\
                         fill_value=0,margins=True)

In [18]:
df_dest_nyc = pd.pivot_table(df_dest,values=['dest_cbg_count'],index=['date_y-m-d'],columns=['dest_sub','dest_subpl'],aggfunc=np.sum,\
                         fill_value=0,margins=True)

In [19]:
df_dest_bor.to_csv(f'output/Destinations/dest_coxsubpl_daily_09-20{y}.csv')
df_dest_nyc.to_csv(f'output/Destinations/dest_nycxsubpl_daily_09-20{y}.csv')

In [20]:
end = time.time()
elapsed = end - start
print(f'Run time - {elapsed} seconds')

Run time - 9877.774066925049 seconds
